# Importing libraries

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm
import torchvision.transforms as tfms
import numpy as np
import time
import csv

# Dataset preparation

In [3]:
data_root = "datasets"
# Run below cell if dataset not downloaded yet

### Run below cell if dataset not downloaded yet

In [ ]:
# data_root = "datasets"

# base_url = "https://graal.ift.ulaval.ca/public/celeba/"

# file_list = [
#     "img_align_celeba.zip",
#     "list_attr_celeba.txt",
#     "identity_CelebA.txt",
#     "list_bbox_celeba.txt",
#     "list_landmarks_align_celeba.txt",
#     "list_eval_partition.txt",
# ]

# # Path to folder with the dataset
# dataset_folder = f"{data_root}/celeba"
# os.makedirs(dataset_folder, exist_ok=True)

# for file in file_list:
#     url = f"{base_url}/{file}"
#     if not os.path.exists(f"{dataset_folder}/{file}"):
#         wget.download(url, f"{dataset_folder}/{file}")

# with zipfile.ZipFile(f"{dataset_folder}/img_align_celeba.zip", "r") as ziphandler:
#     ziphandler.extractall(dataset_folder)

In [4]:
batch_size = 64
image_size = 224
w, h = 218, 178  # the width and the hight of original images before resizing
gender_index = 20  # in the CelebA dataset gender information is the 21th item in the attributes vector.
imagenet_mean = [0.485, 0.456, 0.406]  # mean of the ImageNet dataset for normalizing
imagenet_std = [0.229, 0.224, 0.225]  # std of the ImageNet dataset for normalizing

The running processor is... cuda


In [5]:
transforms = tfms.Compose(
    [
        tfms.Resize((image_size, image_size)),
        tfms.ToTensor(),
        tfms.Normalize(imagenet_mean, imagenet_std),
    ]
)
train_dataset = datasets.CelebA(data_root, split="train", target_type=["attr"], transform=transforms)
valid_dataset = datasets.CelebA(data_root, split="valid", target_type=["attr"], transform=transforms)
test_dataset = datasets.CelebA(data_root, split="test", target_type=["attr"], transform=transforms)

In [6]:
# Only taking part of the dataset to train
train_num = 5000
train_ratio = 0.7

train_subset = Subset(train_dataset, np.arange(1, int(train_num*train_ratio)))
valid_subset = Subset(valid_dataset, np.arange(1, int(train_num*(1-train_ratio))))
test_subset = Subset(test_dataset, np.arange(1, 1000))
train_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# Custom Loss function

In [2]:
class ClassificationRegressionLoss(nn.Module):
    def __init__(self):
        super(ClassificationRegressionLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss()  # size_average=False

    def forward(self, y_pred, y_true):
        loss_cls = self.ce_loss(y_pred, y_true[:, gender_index])  # Cross Entropy Error (for classification)
        return loss_cls

# Model Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("The running processor is...", device)

In [21]:
# Define loss function
criterion = ClassificationRegressionLoss()

In [28]:
model_list = ['inception_v4', 'inception_resnet_v2']
learning_rate_list = [0.01,0.005,0.001]
optimizer_list = ['SGD','RMSprop', 'Adam'] # 'SGD'
momentum_list = [0.0, 0.2, 0.4]

In [29]:
num_epochs = 30

for model in model_list:
    # Initialize the Inception model
    model = timm.create_model(model, pretrained=True, num_classes=2)
    model.to(device)
    break
    
    for optimizer_name in optimizer_list:
        for learning_rate in learning_rate_list:
          for momentum in momentum_list:
        
            if optimizer_name == 'SGD':
              optimizer = optim.SGD(model.parameters(), lr=learning_rate)
            elif optimizer_name == 'RMSprop':
              optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
            elif optimizer_name == 'Adam':
              optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
            log_filename = f'saves/inceptionV4_{optimizer_name}_{momentum}_{learning_rate}.tsv'
            
            with open(log_filename, 'w', newline='') as log_file:
                log_writer = csv.writer(log_file, delimiter='\t')
            
                # Write the header row to the log file
                log_writer.writerow(['epoch', 'time', 'lr', 'loss', 'val_loss'])
            
                for epoch in range(num_epochs):
                    epoch_start_time = time.time()
                    for phase in ['train', 'val']:
                        start_time = time.time()
                        if phase == 'train':
                            model.train()
                            dataloader = train_dataloader
                        else:
                            model.eval()
                            dataloader = valid_dataloader
                
                        running_loss = 0.0
                        corrects = 0
                
                        for inputs, labels in dataloader:
                            inputs = inputs.to(device)
                            labels = labels.to(device)
                
                            optimizer.zero_grad()
                
                            with torch.set_grad_enabled(phase == 'train'):
                                outputs = model(inputs)
                                loss = criterion(outputs, labels)
                
                                if phase == 'train':
                                    loss.backward()
                                    optimizer.step()
                
                            # Calculate the number of correct predictions for binary classification
                            _, preds = torch.max(outputs, 1)
                            
                            # Assuming labels.data is a 2D tensor of shape [batch_size, num_classes], you need to extract the binary class labels
                            # If labels.data contains multiple columns, you can get the binary class labels from one of the columns
                            binary_labels = labels.data[:, gender_index].to(device)
                        
                            # Calculate the number of correct predictions
                            corrects += torch.sum(preds == binary_labels)
                        
                            running_loss += loss.item() * inputs.size(0)
                
                        end_time = time.time()
                        cur_epoch_time = end_time - start_time
            
                        epoch_loss = running_loss / len(dataloader.dataset)
                        
                        if phase == 'train':
                            train_loss = epoch_loss
                        else:
                            val_loss = epoch_loss
                            
                        epoch_acc = corrects.double() / len(dataloader.dataset)
                        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                        print(f'Time taken for {phase} epoch: {cur_epoch_time:.2f} seconds')
                    
                    full_epoch_time = time.time() - epoch_start_time
                    # Record the learning rate
                    current_lr = optimizer.param_groups[0]['lr']
                    log_writer.writerow([epoch + 1, full_epoch_time, current_lr, train_loss, val_loss])  # You need to compute validation loss

                # Save the trained model
                # torch.save(model.state_dict(), 'inceptionv4_celeba.pth')
